In [16]:
from langchain.document_loaders import TextLoader #tectloader allows to load data from different text file
from psutil import cpu_times
from transformers.processing_utils import transformers_module

loader = TextLoader("data/nvda_news_1.txt")
data = loader.load()
print(data[0].metadata)
print(type(loader))
loader.file_path

{'source': 'data/nvda_news_1.txt'}
<class 'langchain_community.document_loaders.text.TextLoader'>


'data/nvda_news_1.txt'

In [8]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader([
    "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
    "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
])

In [13]:
data_1 = loader.load()
len(data_1)

2

In [19]:
data_1[0].metadata

{'source': 'https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html'}

In [22]:
#splitting of text will be done now because a bigger text needs to be splitted into smaller chunks as they is cost associated for calling open ai first huge text data can be splitted and then merging operation can be performed so that if some chunks are there with smaller chunk size will get merge so they bith come near to the limit and thus effiency wil get increased by decreasing the number of chunks opelapping of chnks should also be done so that the next chunk will have some context of previous chunk as well and it make serahcing more error prone
 # so spliiuted can be done using slice operator as well but doing like that may cut words in between so we will use langchain text splitter class for the same


text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.
Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg.
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.

Interstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014.
It has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""





In [24]:
from langchain_text_splitters import CharacterTextSplitter
splitter = CharacterTextSplitter ( separator= "\n",
                                   chunk_size= 200,
                                   chunk_overlap=0
)

chunks = splitter.split_text(text)
len(chunks)

Created a chunk of size 437, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 358, which is longer than the specified 200


7

In [26]:
for chunk in chunks:
    print(len(chunk)) # here some of the chunks are of more size then 200 so we can use recursivetrext spliiter as character split will not work good everytime recursive text spluitter we will give a list of splitters os that text will get break with the first splitter then after splitting by this it will see the size of individual chunks if it is more then 200 then it will use second splitter like this


437
181
197
207
128
357
253


In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter (separators= ["\n\n","\n",".","?"],
                                   chunk_size= 200,
                                   chunk_overlap=0
)

chunks = r_splitter.split_text(text)
print(len(chunks))
for chunk in chunks:
    print(len(chunk))
    print(chunk)

13
104
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
121
. It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine
211
. Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind
1
.
181
Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg.
197
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
112
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm
95
. Principal photography began in l

In [37]:
#if some chunk size is more than 200 then it will split accordingly like 210 then 200 chunk 1 10 chunk 2 like this
#now text splitting is done now vector database to store the chunks and the vector DB used here is faiss )facebook Ai similarity serach) this is a library which allows us to do faster serach into the set of vector which u have

# first the chunks obtained from text splitter will be converted into embeddings and then
#second these embeddings will be stored in a vector DB here its faiss index so this library will have all the embeddings
#as soon as into questions come on the bot UI will convert that question also into embeddings and then give to faiss
#fourth then faiss will perform a vector serach and serach what all embeddings matched with the query embedding and give those vectors


In [ ]:
#next part after vector DB is retrival Q N A with source chain here the relevant chunks will be fetched from teh vectordb using faiss now after getting the chunks will feed those chunks in llm prompt and then ftell teh prompt to find out the answer of question asked using teh below share cunks using this it will solve the problem of cost and save time as well
# this method of combining chunks and given in prompt is called stuff method here the drawback is in case we got many relevant chunks then giving those chunks in prompt will excced the word limit
#so the better approch then stuff method is the map reduce method in this method each chunk will be passed to LLM one by onemeans 4 LLM call and then 4 ansswers will be there and then we can use the final llm call and combine the answer but this method also has drawback like many LLM calls are there